In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
val = pd.read_csv('validation.csv')

In [3]:
train

,Unnamed: 0,Link,Content,Content_cleaned_v1,Content_cleaned_v2,Label
0,22331,https://plo.vn/truy-to-ong-gia-86-tuoi-hiep-da...,Viện KSND TP Hà Nội vừa hoàn tất cáo trạng tru...,viện ksnd tp hà_nội vừa hoàn_tất cáo_trạng tru...,viện ksnd tp hà_nội vừa hoàn_tất cáo_trạng tru...,Phap luat
1,17494,https://vietnamnet.vn/duong-da-chuyen-sau-voi-...,Nám là “cơn ác mộng” với da mặt. Nám xuất hiện...,nám là cơn ác_mộng với da mặt nám xuất_hiện ch...,nám cơn ác_mộng da mặt nám xuất_hiện chủ_yếu t...,Suc khoe
2,6193,https://vietnamnet.vn/gia-vang-mieng-hom-nay-2...,"Mở cửa thị trường hôm nay (20/12), giá vàng mi...",mở_cửa thị_trường hôm_nay giá vàng miếng của c...,mở_cửa thị_trường hôm_nay vàng miếng công_ty t...,Kinh te
3,21758,https://laodong.vn/lao-dong-cuoi-tuan/neu-khon...,"Trong đời sống thường ngày, chúng ta cũng có t...",trong đời_sống thường_ngày chúng_ta cũng có_th...,đời_sống thường_ngày chúng_ta phút giác_ngộ đờ...,Van hoa
4,19942,https://khoahoc.tv/thiet-bi-giup-dieu-khien-ma...,Thiết bị mới có thể hỗ trợ người khuyết tật và...,thiết_bị mới có_thể hỗ_trợ người khuyết_tật và...,thiết_bị hỗ_trợ khuyết_tật phép vướng tay sử_d...,Khoa hoc
...,...,...,...,...,...,...
15906,11964,https://vietnamnet.vn/le-hoi-duong-sach-tet-tp...,Sở Thông tin và Truyền thông TP.HCM báo cáo kế...,sở thông_tin và truyền_thông tp hcm báo_cáo kế...,sở thông_tin truyền_thông tp hcm báo_cáo kết_q...,Giai tri
15907,21575,https://laodong.vn/lao-dong-cuoi-tuan/doc-dao-...,"Ông Nguyễn Minh Nhật (sinh năm 1954, ở Lai Xá)...",ông nguyễn_minh_nhật sinh năm lai_xá chia_sẻ l...,sinh lai_xá chia_sẻ làng lai_xá xưa tên gọi kẻ...,Van hoa
15908,5390,https://vietnamnet.vn/doanh-nghiep-ve-dich-som...,Kỷ lục lịch sử bị xô đổ sớm “Rất đáng tự hào k...,kỷ_lục lịch_sử bị xô đổ sớm rất đáng tự_hào kh...,kỷ_lục lịch_sử xô đổ sớm đáng tự_hào kỷ_lục tậ...,Kinh te
15909,860,https://vietnamnet.vn/hagl-thang-to-viettel-mo...,Chiến thắng của HAGL Đội bóng phố Núi một lần ...,chiến_thắng của hagl đội bóng phố núi một lần ...,chiến_thắng hagl đội bóng phố núi lần nữa khiế...,The thao


In [62]:
X_train, Y_train = train["Content_cleaned_v2"], train["Label"]
X_test, Y_test = test["Content_cleaned_v2"], test["Label"]
X_val, Y_val = val["Content_cleaned_v2"], val["Label"]

In [63]:
Y_train = pd.get_dummies(Y_train)
Y_test = pd.get_dummies(Y_test)
Y_val = pd.get_dummies(Y_val)

In [64]:
# from keras_preprocessing.text import Tokenizer
# import pickle
# from keras_preprocessing.sequence import pad_sequences

# tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')

# tokenizer.fit_on_texts(X_train)
# sequences = tokenizer.texts_to_sequences(X_train)
# with open('tokenizer.pkl', 'wb') as f:
#     pickle.dump(tokenizer, f)


In [65]:
from gensim.models import Word2Vec, KeyedVectors
w2v = Word2Vec.load("../../FeatureEngineering/Vectorize/w2vmodel_skip_256.model")

In [66]:
import pickle
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))

In [67]:
import numpy as np
notin = []
embedding_dim = 256
maxlen = 100
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
print('num_words:', num_words)
for word, i in word_index.items():
    if word in w2v.wv.key_to_index:
            embedding_matrix[i] = w2v.wv[word]
    else:
        notin.append(word)

num_words: 57132


In [68]:
word_index

{'nên': 1,
 'thời_gian': 2,
 'cả': 3,
 'triệu': 4,
 'mức': 5,
 'đầu_tư': 6,
 'chưa': 7,
 'tỉnh': 8,
 'sử_dụng': 9,
 'hoạt_động': 10,
 'lần': 11,
 'nói': 12,
 'lớn': 13,
 'cô': 14,
 'vì': 15,
 'hay': 16,
 'bên': 17,
 'xây_dựng': 18,
 'so': 19,
 'đạt': 20,
 'đối_với': 21,
 'rằng': 22,
 'doanh_nghiệp': 23,
 'điều': 24,
 'ngành': 25,
 'kinh_tế': 26,
 'công_ty': 27,
 'bằng': 28,
 'nếu': 29,
 'tp': 30,
 'thứ': 31,
 'hoặc': 32,
 'thấy': 33,
 'tỷ': 34,
 'giáo_dục': 35,
 'cấp': 36,
 'điểm': 37,
 'tiếp_tục': 38,
 'bạn': 39,
 'gần': 40,
 'con': 41,
 'một_số': 42,
 'tạo': 43,
 'quy_định': 44,
 'đầu': 45,
 'xã_hội': 46,
 'họ': 47,
 'tuổi': 48,
 'y_tế': 49,
 'thông_tin': 50,
 'đưa': 51,
 'học_sinh': 52,
 'thị_trường': 53,
 'tiền': 54,
 'cơ_sở': 55,
 'thi': 56,
 'tốt': 57,
 'dự_án': 58,
 'thêm': 59,
 'học': 60,
 'tuy_nhiên': 61,
 'khoảng': 62,
 'ngân_hàng': 63,
 'vàng': 64,
 'chương_trình': 65,
 'từng': 66,
 'giúp': 67,
 'đặc_biệt': 68,
 'bà': 69,
 'giữa': 70,
 'nghiên_cứu': 71,
 'bệnh': 72,
 'mỗi': 

In [95]:
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
    
    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'att': self.att,
            'ffn': self.ffn,
            'layernorm1': self.layernorm1,
            'layernorm2': self.layernorm2,
            'dropout1': self.dropout1,
            'dropout2': self.dropout2,
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim
        })
        return config
    
    

In [96]:
 # Two seperate embedding layers, one for tokens, one for token index (positions)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim, num_words, embedding_matrix, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
        self.maxlen = maxlen
        self.embed_dim = embed_dim
        self.num_words = num_words
        self.embedding_matrix = embedding_matrix
        self.token_emb = layers.Embedding(num_words, output_dim=embed_dim, input_length=maxlen,weights=embedding_matrix,trainable=False)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    def get_config(self):
        config = super().get_config()
        config.update({
            'maxlen': self.maxlen,
            'embed_dim': self.num_words,
            'num_words': self.num_words,
            'embedding_matrix': self.embedding_matrix,
            'token_emb': self.token_emb,
            'pos_emb': self.pos_emb
        })
        return config

In [97]:
embed_dim = 256  # Embedding size for each token
num_heads = 10    # Number of attention heads
ff_dim = 256    # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen,  embed_dim, num_words, [embedding_matrix])
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(9, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100)]             0         
                                                                 
 token_and_position_embeddin  (None, 100, 256)         14651392  
 g_3 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_3 (Transf  (None, 100, 256)         2761984   
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_3   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_14 (Dropout)        (None, 256)               0   

In [72]:
from keras_preprocessing import sequence
X_train1 = tokenizer.texts_to_sequences(X_train)
X_train2 = sequence.pad_sequences(X_train1, maxlen)

In [73]:
# from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint = ModelCheckpoint('best_trans_skip_256_1.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

In [74]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [75]:
X_val1 = tokenizer.texts_to_sequences(X_val)
X_val2 = sequence.pad_sequences(X_val1, maxlen)

In [98]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0075), loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train2, Y_train, batch_size=64, epochs=1, validation_data=(X_val2, Y_val), callbacks=[early_stopping])
model.save("cc.h5")

249/249 [==============================] - 404s 2s/step - loss: 1.6160 - accuracy: 0.3961 - val_loss: 0.8983 - val_accuracy: 0.6840


In [78]:
X_test1 = tokenizer.texts_to_sequences(X_test)
X_test2 = sequence.pad_sequences(X_test1, 350)

In [79]:
import tensorflow as tf

In [100]:
model_loaded = tf.keras.models.load_model('./cc.h5', custom_objects={'TokenAndPositionEmbedding': TokenAndPositionEmbedding, 'TransformerBlock': TransformerBlock})

TypeError: ('Keyword argument not understood:', 'token_emb')

In [ ]:
loss, accuracy = model_loaded.evaluate(X_test2, Y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

214/214 [==============================] - 18s 79ms/step - loss: 0.2504 - accuracy: 0.9217
Test Loss: 0.25043317675590515
Test Accuracy: 0.9217008948326111


In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()